# 📘 Explainable Stylo Workshop Notebook

Welcome to the Colab notebook for the **Explainable Stylo Workshop**.

This notebook helps you:
- Load the [`cl_explainable_stylo`](https://github.com/remolek/Wshop-ExplainableStylo) package from GitHub
- Prepare to work with textual data stored in folders like `example1/`, `example2/`
- Understand how to explore and use the package interactively


In [ ]:
# 🔧 Setup: Clone the repository and install the package
import os

# Clone the repo only if it's not already cloned
if not os.path.exists("Wshop-ExplainableStylo"):
    !git clone https://github.com/remolek/Wshop-ExplainableStylo.git
%cd Wshop-ExplainableStylo

# Install the package in editable mode
!pip install -e .

## 📦 Import the Package

Now that the package is installed, you can import its modules and use them in the notebook.
Replace `some_module` below with the actual module or function you want to explore.


In [ ]:
# Example: Import a module from cl_explainable_stylo
from cl_explainable_stylo import example_loader  # replace with real module/function

# Help or test it
# help(example_loader)

## 📁 Auto-detect and Preview Textual Data

This cell finds all `example*` folders, lists `.txt` files, and shows a short preview of each.

In [ ]:
import os
from glob import glob

# 🔍 Find all folders starting with "example"
example_dirs = sorted([d for d in os.listdir() if d.startswith("example") and os.path.isdir(d)])
print(f"Detected folders: {example_dirs}")

# 📄 Find and preview all .txt files inside each folder
for folder in example_dirs:
    txt_files = sorted(glob(os.path.join(folder, "*.txt")))
    print(f"\n📁 {folder} contains {len(txt_files)} .txt files:")
    for path in txt_files:
        print(f"  - {os.path.basename(path)}")
        with open(path, "r", encoding="utf-8") as f:
            preview = f.read(200).strip().replace("\n", " ")
        print(f"    Preview: {preview[:150]}{'...' if len(preview) > 150 else ''}")

## 🧪 Your Playground

You can now run your own analyses using functions from `cl_explainable_stylo`, loading texts and experimenting with stylistic explainability.

In [ ]:
# Write your own code here!
# For example, analyze loaded text, vectorize it, or explain features
# from cl_explainable_stylo import analyze_text

# results = analyze_text(text)
# print(results)